In [1]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021 as bbdc
import pandas as pd
from importlib import reload
reload(bbdc)
import tensorflow as tf
from keras import layers
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from matplotlib import colors 
import evaluation.evaluate as evaluate
from tqdm import tqdm

In [2]:
np.random.seed(1)
tf.random.set_seed(1)
pathToDataset = "dataset_fft_mel_l04_o02495_f32/"

In [3]:
model = keras.models.load_model("classification_2d_BBDC_Data_100.h5")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, 32)]        0         
_________________________________________________________________
reshape_2 (Reshape)          (None, None, 32, 1)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, 32, 25)      250       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, None, 32, 25)      5650      
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 32, 25)      100       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, None, 16, 25)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, None, 16, 20)      4520  

# Prediction der Challenge-Daten

In [145]:
pathToDataDir = "../data/"
pathToDataset = "dataset_fft_mel_l04_o02495_f32/"
firstPredictionFile = "../submissions/challenge_submission_t08n03b2_extra_submission7.csv"
firstPrediction = pd.read_csv(firstPredictionFile)
firstPrediction.head()

,filename,onset,offset,event_label,MeanProb,MedianProb
0,10001_mix.wav,2.857143,3.684211,Cheering,0.981393,0.995045
1,10001_mix.wav,5.513784,6.390977,Doorbell,0.887982,0.984081
2,10002_mix.wav,0.426065,2.481203,Camera,0.551233,0.629025
3,10002_mix.wav,2.506266,3.508772,Shatter,0.916703,0.997395
4,10002_mix.wav,5.137845,6.090226,Church_bell,0.341877,0.251322


In [146]:
firstPrediction["numberOfFile"] = firstPrediction.filename.str.split(pat="_", n=1, expand=True)[0].astype(int)
firstPrediction.MedianProb[firstPrediction.numberOfFile<=10500]=1
firstPrediction.head()

<ipython-input-146-346313726371>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firstPrediction.MedianProb[firstPrediction.numberOfFile<=10500]=1


,filename,onset,offset,event_label,MeanProb,MedianProb,numberOfFile
0,10001_mix.wav,2.857143,3.684211,Cheering,0.981393,1.0,10001
1,10001_mix.wav,5.513784,6.390977,Doorbell,0.887982,1.0,10001
2,10002_mix.wav,0.426065,2.481203,Camera,0.551233,1.0,10002
3,10002_mix.wav,2.506266,3.508772,Shatter,0.916703,1.0,10002
4,10002_mix.wav,5.137845,6.090226,Church_bell,0.341877,1.0,10002


In [157]:
ignoredClasses = ['Cough', 'Burping_and_eructation', 'Scratching_(performance_technique)']
predictUntilProb = 0.7
usedProb = "MedianProb"

In [158]:
predictionProbabilities = firstPrediction[usedProb].values
predictionProbabilities[np.isnan(predictionProbabilities)] = 0
sortedIndices = np.argsort(predictionProbabilities)

In [159]:
index  = 0
X = []
indicesToReplace = []
while firstPrediction.iloc[sortedIndices[index]][usedProb]<predictUntilProb:
    row = firstPrediction.iloc[sortedIndices[index]]
    if row.event_label not in ignoredClasses:
        features = np.genfromtxt(pathToDataDir + pathToDataset +"/eval/" + row["filename"].replace(".wav", ".csv"), delimiter=',')
        indices = np.where(np.logical_and(features[:,0]>=row["onset"], features[:,0]<=row["offset"]))[0]
        start = indices[0]
        end = indices[-1]
        while end-start<44:
            start = start-10
            end = end+10
            if end>=len(features):
                end = len(features)
            if start<0:
                start=0
        X.append(features[start:end, 1:])
        indicesToReplace.append(sortedIndices[index])
    index += 1

In [160]:
firstPrediction["oldLabel"] = firstPrediction.event_label
newPrediction = firstPrediction.copy()
for i in range(len(X)):
    newLabel= bbdc.invLabelMap[np.argmax(model.predict(np.array([X[i]])))+1]
    newPrediction.at[sortedIndices[i], "event_label"] = newLabel

In [161]:
print(len(indicesToReplace))

1754


In [162]:
n = np.sum(newPrediction.event_label.values != newPrediction.oldLabel.values)
print("Changed "+str(n)+" Labels")

Changed 1571 Labels


In [163]:
for i in range(10):
    row = newPrediction.iloc[indicesToReplace[i]]
    print("File: "+ row.filename + "  Label was: " +row.oldLabel + "     now is "+row.event_label + "  "+str(row.onset)+" bis "+str(row.offset))

File: 11173_mix.wav  Label was: Fireworks     now is Shatter  0.12531328320802 bis 1.45363408521303
File: 11465_mix.wav  Label was: Fireworks     now is Fireworks  0.952380952380952 bis 1.9047619047619
File: 12143_mix.wav  Label was: Bark     now is Meow  2.68170426065163 bis 4.61152882205514
File: 12619_mix.wav  Label was: Camera     now is Bark  1.70426065162907 bis 2.65664160401002
File: 10802_mix.wav  Label was: Cheering     now is Bark  5.13784461152882 bis 6.11528822055138
File: 11072_mix.wav  Label was: Camera     now is Meow  5.11278195488722 bis 6.59147869674185
File: 12495_mix.wav  Label was: Church_bell     now is Bark  5.03759398496241 bis 6.8421052631579
File: 12122_mix.wav  Label was: Church_bell     now is Meow  7.44360902255639 bis 8.7468671679198
File: 10768_mix.wav  Label was: Shout     now is Meow  0.25062656641604 bis 2.13032581453634
File: 11132_mix.wav  Label was: Meow     now is Cough  2.98245614035088 bis 3.88471177944862


In [164]:
for i in range(1,20):
    row = newPrediction.iloc[indicesToReplace[-i]]
    print("File: "+ row.filename + "  Label was: " +row.oldLabel + "     now is "+row.event_label + "  "+str(row.onset)+" bis "+str(row.offset))

File: 11515_mix.wav  Label was: Camera     now is Camera  5.38847117794486 bis 6.41604010025063
File: 12651_mix.wav  Label was: Fireworks     now is Fireworks  3.80952380952381 bis 4.53634085213033
File: 11215_mix.wav  Label was: Cheering     now is Cheering  5.41353383458647 bis 6.34085213032581
File: 12347_mix.wav  Label was: Fireworks     now is Fireworks  9.07268170426065 bis 9.47368421052632
File: 12136_mix.wav  Label was: Fireworks     now is Fireworks  1.52882205513784 bis 3.25814536340852
File: 12817_mix.wav  Label was: Fireworks     now is Fireworks  0.75187969924812 bis 1.70426065162907
File: 11208_mix.wav  Label was: Meow     now is Meow  2.68170426065163 bis 4.63659147869674
File: 10809_mix.wav  Label was: Church_bell     now is Church_bell  2.5062656641604 bis 3.38345864661654
File: 10858_mix.wav  Label was: Cheering     now is Cheering  4.53634085213033 bis 5.51378446115288
File: 11787_mix.wav  Label was: Doorbell     now is Doorbell  2.93233082706767 bis 3.90977443609023

In [165]:
newPrediction.to_csv("../submissions/submission17/submission7_extra_changedUnsecure.csv", index=False)
newPrediction.head()

,filename,onset,offset,event_label,MeanProb,MedianProb,numberOfFile,oldLabel
0,10001_mix.wav,2.857143,3.684211,Cheering,0.981393,1.0,10001,Cheering
1,10001_mix.wav,5.513784,6.390977,Doorbell,0.887982,1.0,10001,Doorbell
2,10002_mix.wav,0.426065,2.481203,Camera,0.551233,1.0,10002,Camera
3,10002_mix.wav,2.506266,3.508772,Shatter,0.916703,1.0,10002,Shatter
4,10002_mix.wav,5.137845,6.090226,Church_bell,0.341877,1.0,10002,Church_bell


In [166]:
predictionForChallenge = newPrediction.drop(columns=['MeanProb', "MedianProb", "numberOfFile", "oldLabel"])
predictionForChallenge.to_csv("../submissions/submission17/submission7_extra_changedUnsecure_forChallenge.csv", index=False)
predictionForChallenge.head()

,filename,onset,offset,event_label
0,10001_mix.wav,2.857143,3.684211,Cheering
1,10001_mix.wav,5.513784,6.390977,Doorbell
2,10002_mix.wav,0.426065,2.481203,Camera
3,10002_mix.wav,2.506266,3.508772,Shatter
4,10002_mix.wav,5.137845,6.090226,Church_bell
